# 0.4.0 Building a "fake news" classifier

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import re

from nltk import pos_tag, ne_chunk_sents
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from gensim.corpora.dictionary import Dictionary

from gensim.models.tfidfmodel import TfidfModel

import spacy

# from polyglot.text import Text

import itertools
from collections import defaultdict

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from utils.info import article_m3, article_f
import utils.paths as path
from utils.paths2 import direcciones, direcciones_cursos

## 0.4.2 Which possible features?

Which of the following are possible features for a text classification problem?

R:/ All of the above.

## 0.4.3 Training and testing

What datasets are needed for supervised learning?

R:/ Both training and testing data.

## 0.4.5 CountVectorizer for text classification